# Acrobot
---
>단순한 환경이지만 cartpole 과 많이 다르다. 우선 오랫동안 버티는 환경과 달리 terminal state 로 빠르게 가는 것이 목적이다. 동시에 제한시간도 있다. 시간제한을 넘기면 아예 가치가 없는 episode 가 되어버린다. 이런 경험은 그냥 버려야한다. 그래서 최대한 많은 랜덤시도를 해야한다. 당연히 시간이 오래걸린다. 실패가 아닌 성공으로부터 학습을 하므로 우선 성공을 하는것이 중요하다. 또한 이러한 이유때문에 episodic 하게 학습을 해야한다.


* state : 조인트 2 개의 sin, cos, 각속도 - continuous
* action : 조인트 사이에 힘을 +1, 0, -1
* reward : 매 프레임마다 -1, 500 프레임이 넘어가면 자연스럽게 종료
---

In [165]:
%matplotlib notebook  
import tensorflow as tf
import gym
import numpy as np
import random
import matplotlib.pyplot as plt

In [166]:
env = gym.make('Acrobot-v1')

In [167]:
size_in = env.observation_space.shape[0]
size_out = env.action_space.n
size_w1 = 8
size_w2 = 12
size_w3 = 6

lr = .1
total_episode = 1000
epsilon = 1
gamma = .95

reward = tf.placeholder(tf.float32)
STATE_IN = tf.placeholder(tf.float32, [None, size_in])
W_1 = tf.Variable(tf.random_normal([size_in, size_w1],stddev=.01), name='W_1')
W_2 = tf.Variable(tf.random_normal([size_w1, size_w2],stddev=.01), name='W_2')
W_3 = tf.Variable(tf.random_normal([size_w2, size_w3],stddev=.01), name='W_3')
out = tf.Variable(tf.random_normal([size_w3, size_out],stddev=.01), name='W_out')

In [168]:
L_1 = tf.matmul(STATE_IN, W_1)
L_2 = tf.matmul(L_1, W_2)
L_3 = tf.matmul(L_2, W_3)
L_out = tf.tanh(tf.matmul(L_3, out)) ##활성화함수가 tanh - 

# simple policy gradient 학습 - likelihood x Q
loss = tf.reduce_mean(tf.log(L_out)*reward)
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [169]:
def discounted_reward(r):
    dr = np.zeros_like(r)
    sum_r = 0
    
    for i in reversed(range(0, r.size)):
        sum_r = gamma*sum_r + r[i]
        dr[i] = sum_r  
    
    return dr

In [170]:
saver = tf.train.Saver()

with tf.Session() as sess:
    
    tf.global_variables_initializer().run()
    state_buffer = []
    reward_buffer = []
    ep = 0
    state = env.reset()
    reward_sum = 0
    step = 0
    is_render = False
    mean_ep_reward = 0
    is_trained = False
    
    while not is_trained:
        done = False
        
        while not done:
            
            step = step + 1
            state = np.reshape(state, [1, size_in])
            state_buffer.append(state)

            if(random.random() < epsilon):
                action = env.action_space.sample()
            else:
                pi = sess.run(L_out, feed_dict={STATE_IN:state})
                action = np.argmax(pi)

            new_state, r, d, _ = env.step(action)
            state = new_state
            reward_sum += r
            reward_buffer.append(r)

            if d:
                if reward_sum == -500:
                  #epsilon = 1
                    pass
                else:
                    ep += 1
                    mean_ep_reward += reward_sum
                    
                    if(ep % 10 == 0):
                        mean_ep_reward = mean_ep_reward / 10
                        print('------------ {} 평균 : {} ------'.format(ep/10,mean_ep_reward))
                        #cartpole과는 다르게 제한시간 내로 목표에 도달하지 못하면 실패한 것으로 
                        # 학습 종료
                            
                        mean_ep_reward = 0
                        
                    if(epsilon < .2):
                        epsilon = .2
                    else:
                        epsilon *= .99
                    
                    '''
                    성공의 직전 행동만 학습하는 trick
                    '''
                    try:
                        state_buffer = state_buffer[len(state_buffer) - 150:]
                        reward_buffer = reward_buffer[len(reward_buffer) - 150:]

                    except:
                        state_buffer = state_buffer[len(state_buffer) - 50:]
                        reward_buffer = reward_buffer[len(reward_buffer) - 50:]
                    
                    
                    eps = np.vstack(state_buffer)
                    epr = np.vstack(reward_buffer)
                    epr = discounted_reward(epr)
                    
                    sess.run(train, feed_dict={STATE_IN:eps, reward:epr})                    
                    print(ep, reward_sum, epsilon)

                state_buffer, reward_buffer = [],[]                
                reward_sum = 0
                done = True
                state = env.reset()
                step = 0

    done = False

1 -438.0 0.99
2 -464.0 0.9801
3 -296.0 0.9702989999999999
4 -306.0 0.96059601
5 -336.0 0.9509900498999999
6 -401.0 0.9414801494009999
7 -408.0 0.9320653479069899
8 -491.0 0.92274469442792
9 -475.0 0.9135172474836407
------------ 1.0 평균 : -409.5 ------
10 -480.0 0.9043820750088043
11 -285.0 0.8953382542587163
12 -468.0 0.8863848717161291
13 -439.0 0.8775210229989678
14 -466.0 0.8687458127689781
15 -376.0 0.8600583546412883
16 -363.0 0.8514577710948754
17 -337.0 0.8429431933839266
18 -297.0 0.8345137614500874
19 -396.0 0.8261686238355865
------------ 2.0 평균 : -365.0 ------
20 -223.0 0.8179069375972307
21 -468.0 0.8097278682212583
22 -314.0 0.8016305895390458
23 -210.0 0.7936142836436553
24 -410.0 0.7856781408072188
25 -310.0 0.7778213593991465
26 -246.0 0.7700431458051551
27 -256.0 0.7623427143471035
28 -239.0 0.7547192872036325
29 -350.0 0.7471720943315961
------------ 3.0 평균 : -305.1 ------
30 -248.0 0.7397003733882802
31 -228.0 0.7323033696543974
32 -365.0 0.7249803359578534
33 -219.0

KeyboardInterrupt: 